# MINIMIZE_MULTI

## Overview
The `MINIMIZE_MULTI` function finds the minimum of a (possibly multivariate) mathematical function using `scipy.optimize.minimize`. This is useful for business users in Excel who need to optimize costs, maximize efficiency, or find the best value for a given scenario involving multiple variables.

This function leverages numerical optimization algorithms from `scipy.optimize.minimize` to solve problems of the form:

$\min_{x \in \mathbb{R}^n} f(x)$

where $f(x)$ is a user-supplied function of $n$ variables, and $x$ is a vector of decision variables. The user provides $f(x)$ as a Python expression in terms of `x`, e.g., `'x[0]**2 + x[1]**2 + 3*x[0]'`. The function supports optional bounds, enabling the solution of both unconstrained and bounded optimization problems. The underlying algorithms include BFGS, SLSQP, COBYLA, and others, which use gradient-based or simplex methods to efficiently search for the minimum.

For example, the Rosenbrock function, a classic test problem for optimization algorithms, is defined as:

$f(x) = (1 - x_0)^2 + 100(x_1 - x_0^2)^2$

The function can handle problems with or without bounds, making it flexible for a wide range of business and engineering applications.

## Usage
To use the `MINIMIZE_MULTI` function in Excel, enter it as a formula in a cell, specifying your function expression and any optional arguments as needed:

```excel
=MINIMIZE_MULTI(func_expr, x_zero, [bounds], [method])
```

## Arguments
| Argument      | Type           | Required | Description                                                        | Example                |
|---------------|----------------|----------|--------------------------------------------------------------------|------------------------|
| func_expr     | string         | Yes      | The function to minimize, as a string.                             | "x[0]**2 + x[1]**2 + 3*x[0]" |
| x_zero        | 2D list        | Yes      | Initial guess for the variables, as a 2D list.                     | `{-1,2}`                 |
| bounds        | 2D list/None   | No       | Optional bounds for variables, as a 2D list of (min, max) pairs.   | `{0,10;0,5}`             |
| method        | string/None    | No       | Optional optimization method.                                      | "BFGS"                |

## Returns
| Returns   | Type           | Description                                                        | Example                |
|-----------|----------------|--------------------------------------------------------------------|------------------------|
| result    | 2D list        | [[x0, x1, ..., fun]]: x-values of minimum and minimum value.          | `[[1.0, 1.0, 0.0]]`        |
| error     | string         | Error message if input is invalid or an error occurs.               | "Error during minimization: ..." |

## Examples

### Minimize Rosenbrock's Function (2 variables)
**Business Context:**
A data scientist wants to find the minimum of the Rosenbrock function, a classic test problem for optimization algorithms: $f(x) = (1 - x_0)^2 + 100(x_1 - x_0^2)^2$.

**Sample Input Data:**
- Function expression: '(1 - x[0])**2 + 100*(x[1] - x[0]**2)**2'
- Initial guess: `{-1,2}`

```excel
=MINIMIZE_MULTI("(1 - x[0])**2 + 100*(x[1] - x[0]**2)**2", {-1,2})
```
**Sample Output:**
`[[1.0, 1.0, 0.0]]`

### Minimize with Bounds
**Business Context:**
A logistics analyst wants to minimize $f(x) = (x_0-5)^2 + (x_1-2)^2$, with $x_0 \in [0, 10]$ and $x_1 \in [0, 5]$.

**Sample Input Data:**
- Function expression: "(x[0]-5)**2 + (x[1]-2)**2"
- Initial guess: `{1,1}`
- Bounds: `{0,10;0,5}`

```excel
=MINIMIZE_MULTI("(x[0]-5)**2 + (x[1]-2)**2", {1,1}, {0,10;0,5})
```
**Sample Output:**
`[[5.0, 2.0, 0.0]]`

### Invalid Input
**Business Context:**
A user enters an invalid function expression or wrong argument type.

**Sample Input Data:**
- Function expression: 'x[0]***2 + x[1] + 2'
- Initial guess: `{0,0}`

```excel
=MINIMIZE_MULTI("x[0]***2 + x[1] + 2", {0,0})
```
**Sample Output:**
"Error during minimization: ..." or "func_expr must be a string."

In [ ]:
from scipy.optimize import minimize
import math

def minimize_multi(func_expr, x_zero=None, bounds=None, method=None):
    """
    Minimizes a function using scipy.optimize.minimize.
    Args:
        func_expr (str): A string representing the function to minimize, e.g., 'x[0]**2 + x[1]**2'.
        x_zero (2D list, required): Initial guess for the variables, as a 2D list (e.g., [[0, 0]]).
        bounds (2D list or None): Bounds for variables, as a 2D list (e.g., [[min1, max1], [min2, max2]]).
        method (str or None): Optimization method supported by scipy.optimize.minimize.
    Returns:
        2D list: [[x0, x1, ..., fun]] where x is the location of minimum and fun is the minimum value, or a string with an error message
    """
    # Input validation and normalization
    if not isinstance(func_expr, str):
        return "func_expr must be a string."
    if x_zero is None or not (isinstance(x_zero, list) and len(x_zero) > 0 and isinstance(x_zero[0], list)):
        return "x_zero (initial guess) must be provided as a 2D list, e.g., [[0, 0]]."
    x0_row = x_zero[0]
    if not all(isinstance(v, (int, float)) for v in x0_row):
        return "x_zero (initial guess) must be a 2D list of numbers."
    bounds_list = None
    if bounds is not None:
        if not (isinstance(bounds, list) and all(isinstance(b, (list, tuple)) and len(b) == 2 for b in bounds)):
            return "bounds must be a 2D list of (min, max) pairs or None."
        bounds_list = [tuple(b) for b in bounds]
    method_str = None
    if method is not None:
        # Accept either a string or a 2D list with a string
        if isinstance(method, list):
            if len(method) > 0 and isinstance(method[0], list):
                method_str = method[0][0] if len(method[0]) > 0 else None
            elif len(method) > 0 and isinstance(method[0], str):
                method_str = method[0]
            else:
                return "method must be a string, 2D list, or None."
        elif isinstance(method, str):
            method_str = method
        else:
            return "method must be a string, 2D list, or None."
    if 'x' not in func_expr:
        return "Function expression must contain the variable 'x'."
    def func(x):
        try:
            return eval(func_expr, {"x": x, "math": math})
        except Exception as e:
            return f"Error evaluating function expression: {str(e)}"
    kwargs = {}
    if bounds_list is not None:
        kwargs['bounds'] = bounds_list
    if method_str is not None:
        kwargs['method'] = method_str
    try:
        result = minimize(func, x0_row, **kwargs)
        if not hasattr(result, 'x') or not hasattr(result, 'fun'):
            return "Error during minimization: Invalid result object."
        if not result.success or not isinstance(result.fun, (int, float)) or result.fun == float('inf'):
            msg = getattr(result, 'message', None)
            if msg:
                return f"Error during minimization: {msg}"
            return "Error during minimization: Optimization failed."
        x_list = [float(xi) for xi in result.x]
        return [x_list + [float(result.fun)]]
    except ValueError as ve:
        return str(ve)
    except Exception as e:
        return f"Error during minimization: {str(e)}"

In [ ]:
%pip install -q ipytest
import ipytest
ipytest.autoconfig()

def test_demo_quadratic_2d_unbounded():
    result = minimize_multi(
        "x[0]**2 + x[1]**2 + 3*x[0]",
        [[0, 0]],
        None,
        None
    )
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert len(result[0]) == 3
    assert all(isinstance(x, float) for x in result[0])

def test_demo_bounded_minimum_2d():
    result = minimize_multi(
        "(x[0]-5)**2 + (x[1]-2)**2",
        [[1, 1]],
        [[0, 10], [0, 5]],
        None
    )
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert len(result[0]) == 3
    assert all(isinstance(x, float) for x in result[0])

def test_demo_invalid_expression_2d():
    result = minimize_multi(
        "x[0]***2 + x[1] + 2",
        [[0, 0]],
        None,
        None
    )
    assert isinstance(result, str) and ("error" in result.lower() or "must be" in result.lower() or "function expression must contain" in result.lower())

def test_demo_with_method_cobyla():
    result = minimize_multi(
        "(x[0]-1)**2 + (x[1]-2)**2",
        [[0, 0]],
        None,
        [["COBYLA"]]
    )
    assert isinstance(result, list)
    assert len(result) == 1
    assert isinstance(result[0], list)
    assert len(result[0]) == 3
    assert all(isinstance(x, float) for x in result[0])

ipytest.run()

In [ ]:
# Interactive Demo
import gradio as gr

examples = [
    [
        "(1 - x[0])**2 + 100*(x[1] - x[0]**2)**2",  # func_expr (Rosenbrock)
        [[-1, 2]],                                   # x_zero
        [[0, 0], [0, 0]],                            # bounds (set to zeros for demo, will be treated as None)
        [["BFGS"]],                                 # method (set to BFGS for demo)
    ],
    [
        "(x[0]-5)**2 + (x[1]-2)**2",
        [[1, 1]],
        [[0, 10], [0, 5]],
        [["SLSQP"]],
    ]
]

demo = gr.Interface(
    fn=minimize_multi,
    inputs=[
        gr.Textbox(label="Function Expression", value="(1 - x[0])**2 + 100*(x[1] - x[0]**2)**2"),
        gr.Dataframe(headers=["x0", "x1"], label="Initial Guess (x_zero)", row_count=1, col_count=2, type="array", value=[[-1, 2]]),
        gr.Dataframe(headers=["min", "max"], label="Bounds (optional)", row_count=2, col_count=2, type="array", value=[[0, 0], [0, 0]]),
        gr.Dataframe(headers=["method"], label="Method (optional)", row_count=1, col_count=1, type="array", value=[["BFGS"]]),
    ],
    outputs=gr.Dataframe(headers=["x0", "x1", "Minimum Value"], label="Result"),
    examples=examples,
    description="Find the minimum of a multivariate function using scipy.optimize.minimize. All arguments must be provided as 2D lists or scalars. Optional arguments can be left empty (as blank DataFrames or lists).",
    flagging_mode="never",
)
demo.launch()